[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/abdn-cs3033-ai/practicals/blob/main/week04/tutorial3-local-search-extra.ipynb)

# CS3033: Artificial Intelligence

## Tutorial 03: Local Search Algorithms - Evolutionary Algorithms

#### Stewart Anderson

**PLEASE READ** 

This tutorial does not contain a lot of boilerplate code as this is designed to aid you in understanding how to develop an evolutionary algorithm from the ground up. To successfully complete this task, you should already have experience with object-oriented design in Python. Additionally, you may need to create additional code blocks throughout the notebook to test your implementation; conduct some independent research on the problem; or carry out independent research on any of the operations or methods described below. Keep in mind that this extra tutorial is intended to be challenging, and future components within this tutorial will depend on code from previous code blocks; so you are actively encouraged to ask for help if required!

You are welcome to use whichever imports you require for this tutorial to make this easier for you; however you are recommended against using imports which handle core components of evolutionary computation (e.g., `DEAP`, `LEAP`, `evolutionary-algorithm`, ...) as they defeat the purpose of what this tutorial sets out to achieve. You do not necessarily have to write this tutorial in Python if you don't want to either. There should be sufficient information in this tutorial to effectively write this in any language you like!

In order to run this tutorial in Google Colab, you will need to download the auxiliary files from GitHub into your notebook, which we do with Jupyter's shell commands (if you downloaded the entire repo, the code below is not necessary).

In [ ]:
try:
    import google.colab
    print("We are in Google colab, we need to clone the repo")
    !git clone https://github.com/abdn-cs3033-ai/practicals.git
    %cd practicals/week04
except:
    print("Not in colab")

%pip install matplotlib

### Imports

Feel free to add any imports you wish to use throughout the tutorial. Some have been provided for you!

Remember that if you add any imports here, you will be required to re-run the code block to access their functionality.

In [ ]:
import random
import math
import matplotlib.pyplot as plt

## Evolutionary Algorithms & The Travelling Salesman Problem
[Evolutionary Algorithms](https://en.wikipedia.org/wiki/Evolutionary_algorithm) (EAs) represent a class of optimisation and search algorithms inspired by the biological evolution process. These algorithms aim to discover solutions to complex problems where traditional methods may prove impractical or ineffective. EAs function by maintaining a population of potential solutions and iteratively evolving them over generations. Each generation involves selecting chromosomes with advantageous traits, applying genetic operators like mutation and crossover to generate new chromosomes, and substituting less fit chromosomes with the newly created ones. This iterative process continues until a satisfactory solution is generated, or a predefined stopping criterion is satisfied. EAs have found successful applications in various domains, including engineering, finance, and machine learning.

The [Travelling Salesman Problem](https://en.wikipedia.org/wiki/Travelling_salesman_problem) (TSP) is a classic combinatorial optimisation challenge. In the TSP, the goal is to find the shortest possible route that allows a salesperson to visit a set of locations exactly once and return to the starting city. This problem has real-world applications in logistics, transportation, and circuit design. The TSP is NP-Hard, meaning that as the number of locations increases, finding the optimal solution becomes increasingly complex and time-consuming. Various algorithms are employed to efficiently approximate solutions to the TSP, enabling practical use in route planning and optimisation scenarios.

The mathematical optimisation of this problem can be formalised as:
$$f:A\mapsto\mathbb{R}^{+}$$
$$\exists x_0\in A\,\vert\,f(x_0) < f(x)\cdot \forall x\in A$$
A function $f$, maps the fitness values of each of the chromosomes $A$ to the set of real numbers. We then hypothesise the existence of some optimal value $x_0$ which has the lowest fitness value of all possible $x$ in $A$, where $x$ is the fitness value of an individual chromosome.

### Problem Representation

In each of the problem files, we have a series of locations, which contain an ID, an X coordinate, and a Y coordinate; all of which are separated by a space. The relevant TSP files have also been downloaded, and are kept in the `tsp/` directory if you wish to take a look at them, and are described in the below table. **NOTE:** The files do *not* contain a header row.

| Location ID | Location X | Location Y |
|------------:|------------|------------|
|     1       |  x<sub>1</sub> |  y<sub>1</sub> |
|     2       |  x<sub>2</sub> |  y<sub>2</sub> |
|     3       |  x<sub>3</sub> |  y<sub>3</sub> |
|     ...     |  ...           |  ...           |

You should design the relevant `Location` class below to accurately represent a location (described by the table above) -- which should make implementing a `Chromosome` class easier later. **REMEMBER:** Once you create a location object, you do not have to modify its `id`, `x`, or `y` fields, so making this class immutable would be ideal!

In [ ]:
class Location:

    def __init__(self, id, x, y):
        """
            Implement the constructor.
        """

    def __str__(self):
        """
            Implement the string representation of the object.
        """
    
    def id(self):
        """
            Implement the ID getter.
        """
    
    def x(self):
        """
            Implement the X getter.
        """
    
    def y(self):
        """
            Implement the Y getter.
        """
        

### The Chromosome
Now that you have designed the representation of `Location`s, your `Chromosome` should be a sequence of `Location`s that defines the order in which the salesperson visits the locations. It represents a possible route or tour that starts and ends at the same city, satisfying the requirement to visit each city exactly once.

Your `Chromosome` will require the following fields:

1. `fitness`: This should be a real value, as your fitness function will return a real value.

2. `locations`: This is a collection of instantiations of the `Location` class.

Your `Chromosome` will also require:

1. An empty constructor, initialising the `fitness` value to be 0 and the `locations` list to be empty.

2. A deep copy constructor, providing the ability to copy the `fitness` and `locations` from any given `Chromosome`.

3. A way to add a `Location` object, or series of `Location` objects to the `Chromosome`.

4. A fitness function, which will be the Euclidean Distance of a given `location` to another `location` expressed in two dimensions:
$$D(l_1,l_2)=\sqrt{(x_{l_2} - x_{l_1})^2 + (y_{l_2} - y_{l_1})^2}$$

5. A method to shuffle the contents of the `locations` list.

7. A method for getting the string representation of the `Chromosome` object.

You are welcome to add more functionality to the `Chromosome` class should you wish to do so.

In [ ]:
class Chromosome:

    def __init__(self, chromosome=None):
        """
            Implement the empty Chromosome constructor.
            Implement the copy Chromosome constructor.
        """

    def add_location(self, location):
        """
            Implement the add_location method.
        """

    def evaluate(self):
        """
            Implement the evaluation function
        """


    def shuffle(self):
        """
            Implement the shuffle method.
        """

    def __str__(self):
        """
            Implement the string representation of the object.
        """

### Loading a Problem
Now that you've successfully defined your `Location` and `Chromosome` classes, it's time to explore how to implement a problem. As previously mentioned, you can find the problems stored in the `tsp/` directory. The available problems are:
+ **berlin52.tsp**: This problem corresponds to 52 points of interest in Berlin.
+ **burma14.tsp**: This problem relates to 14 points of interest in Burma.
+ **dj38.tsp**: This problem pertains to 38 points of interest in Djibouti.
+ **eil51.tsp**: This is a synthetic dataset comprising 51 locations.
+ **lin318.tsp**: This is also a synthetic dataset containing 318 locations.

Your `Problem` class should contain a static `load_problem` method which takes in a single parameter: `file`, which is the string name of the `.tsp` file to load in. 

In case you haven't encountered a static method before, static methods belong to a class instead of an independent object, meaning you are not required to instantiate an object of a class to access the method. **BE CAREFUL:** Since static methods and fields belong to a class instead of an object, each object of a class that contains a static field or method will contain the same value. You could also encounter something called _static abuse_, which is where you over-use the static property to reduce object-oriented code and is generally regarded as bad practice. Essentially; only use it where absolutely necessary!

The `load_problem` method's functionality should be as follows:
+ This method should begin by creating an empty `Chromosome` object.
+ It should then iterate over each line in the file, assuming that each line represents a `Location` -- an `id`, `x` coordinate, and `y` coordinate, separated by a space.
+ This `Location` should then be added to the `Chromosome` object.
+ After adding all `Location`s to the `Chromosome` object, you should shuffle the `locations` list.
+ Lastly, return the `Chromosome` object, which should now contain a permutation of all locations in the file.

In [ ]:
class Problem:
    
    @staticmethod
    def load_problem(file):
        chromosome = Chromosome()

        """
            Implement the load_problem method.
        """

        return chromosome

### Creating the Evolutionary Algorithm
An evolutionary algorithm's approach to solving the travelling salesman problem involves several key steps:

1. **Population Initialisation:** A population of potential routes, often represented as permutations of locations, is created. Each route represents a potential solution to the TSP.

2. **Evaluation:** The fitness of each route is assessed, usually by calculating the total distance required to traverse the locations along that route.

3. **Selection:** Routes are chosen for reproduction based on their fitness. Routes with shorter total distances (i.e., better solutions) have a higher chance of being selected.

4. **Recombination (Crossover):** Pairs of routes are combined to create offspring routes. This process mimics genetic recombination in biology. For example, a crossover operation might involve merging two parent routes to create a new route with characteristics inherited from both parents. Be weary, however; as not all crossover operators are applicable to all problems. This is also known as the exploitation operator.

5. **Mutation:** Occasionally, random changes are introduced into routes to encourage exploration of new solutions. This process emulates genetic mutations. This is known as the exploration operator.

6. **Replacement:** Less fit routes are replaced by the newly generated offspring, ensuring that the population evolves over time towards better solutions.

7. **Termination Criterion:** The algorithm continues evolving the population through multiple generations until a termination criterion is met. This could be a specific number of generations, a time limit, or reaching a satisfactory solution.

These steps can be illustrated below, depicting their connection to each other.
<div style="text-align: center;"><img src="img/evolutionary-cycle.png" style="max-width: 60%;" /></div>

#### Parameters
Now that we have begun to consider the possibility of implementing an evolutionary algorithm, we need to consider what the parameters will be. This is so we can not only make our results replicable, but also allow for modularity in our solution.

If you elect to have more parameters in your implementation for experimentation purposes, please feel free to add more parameters as you require them.

In [ ]:
class Parameters:
    
    # Problem
    problem = 'burma14.tsp'

    # Population size
    population_size = 30

    # Number of generations
    num_generations = 5000

    # Initialisation method (e.g., 'random', 'greedy')
    initialisation_method = 'augmented'

    # Selection mechanism (e.g., 'roulette', 'tournament', 'rank_based')
    selection_method = 'roulette'

    # Crossover rate (probability of crossover)
    crossover_rate = 0.7

    # Mutation rate (probability of mutation)
    mutation_rate = 0.1

    # Termination criterion (e.g., 'max_generations', 'target_fitness', 'no_improvement')
    termination_criterion = 'max_generations'

    # Crossover operator (e.g., 'one_point', 'two_point', 'uniform')
    crossover_method = 'ordered'

    # Mutation operator (e.g., 'swap', 'invert', 'random')
    mutation_method = 'swap'

    # Replacement operator (e.g., 'worst', 'generation')
    replacement_method = 'steady_state'

    # The ratio to keep the best n% of the population (value between 0 and 1)
    elitism_ratio = 0.4

    # Seed for random number generation (for reproducibility)
    random_seed = 42
    

### Checking it Works
Before we can move forward, we need to check if this implementation works! You can now construct a short hill-climbing algorithm to check that your implementation works:
+ Begin by creating a singular `Chromosome` object, and use your `load_problem` method to initialise it with a permutation of the possible routes, referencing the problem file you have chosen from the `Parameters` class.
+ Enter a loop with a fixed number of generations -- try 5,000 to start (modify this in the `Parameters` class, and refer to your `num_generations` property).
+ Shuffle the `Chromosome` object.
+ Check if the current `Chromosome` object's fitness is lower than the previous best fitness.
    + If it is
        + Announce a new best fitness found and print this value.
        + Set the best fitness value to be the current fitness.

After you have finished this


In [ ]:
class HillClimber:
    def __init__(self):
        """
            Implement the constructor.
        """
    
    def hill_climber(self):
        """
            Implement the hill climber method.
        """


In [ ]:
climber = HillClimber()

climber.hill_climber()

Assuming your code above works; what do you notice about your results:
+ If your hill-climber runs for 5,000 generations?
+ If your hill-climber runs for 10,000 generations?
+ If your hill-climber runs for 20,000 generations?
+ If your hill-climber runs indefinitely?
+ Do you believe you get the most optimal solution?
    + Why?
+ Do you believe you get anywhere near the optimal solution?
    + Why?

#### Initialisation
When embarking on the journey of creating and fine-tuning an EA, one of the fundamental steps is initialisation. The way you initialize your population sets the stage for the entire evolutionary process, influencing the algorithm's performance and ability to find optimal solutions.

Initialisation is akin to setting the stage for a grand performance. In the world of evolutionary computation, it's the act of creating the initial set of potential solutions or `Chromosome`s, often referred to as the population. These `Chromosome`s undergo a series of genetic operations such as selection, crossover, and mutation to evolve towards better solutions over generations.

Getting the initialisation right is a critical aspect of EA design. Poor initialisation can lead to premature convergence or stagnation, where the algorithm gets stuck in suboptimal solutions. Conversely, a well-crafted initialisation strategy can jumpstart the evolutionary process, helping the algorithm explore the solution space effectively.

**Common Initialisation Strategies:**

1. **Random Initialisation:** This basic strategy involves creating `Chromosome`s with random values. While simple, it can be surprisingly effective in some cases, especially when the solution space is not well understood.

2. **Augmented Initialisation:** This method involves generating a larger population through the random shuffling of the problem. After evaluating their fitness values, the population size is reduced to its intended size. While theoretically resulting in a fitter population, this isn't guaranteed.

3. **Greedy Initialisation:** In this approach, `Chromosome`s are generated using heuristics or greedy algorithms to construct relatively good initial solutions. It's particularly useful when prior knowledge about the problem domain is available.

4. **Seed Initialisation:** Sometimes, known good solutions or seeds are used as a starting point. This approach is valuable when you have access to or prior experience with solutions that are close to optimal.

Choosing the right initialisation strategy is a delicate balance between exploration and exploitation. You aim to create a diverse population that covers various regions of the solution space while not straying too far from potentially good solutions.

In [ ]:
class Initialisation:

    @staticmethod
    def random():
        population = []
        """
            Implement the random initialisation method.
        """
        return population

    @staticmethod
    def augmented():
        augmented_population = []
        """
            Implement the augmented population method.
        """
        return augmented_population
    
# Feel free to add any additional operators below.

#### Selection
Selection is a cornerstone of an EA, playing a pivotal role in shaping the genetic makeup of future generations. It's the process by which `Chromosome`s are chosen from the current population to serve as parents for the creation of offspring in the next generation. This critical step is responsible for directing the algorithm's search toward better solutions.

Imagine nature's process of selecting organisms with favourable traits for reproduction. Similarly, in EAs, selection identifies and favours `Chromosome`s with higher fitness values—those closer to the optimal solution. By emphasizing the reproduction of fit `Chromosome`s, selection gradually refines the population, guiding it towards improved solutions over successive generations.

**Common Selection Techniques:**

1. **Roulette Wheel Selection:** This technique assigns a slice of a roulette wheel to each individual based on their fitness. The wheel is then spun, and `Chromosome`s with larger slices have a higher chance of being selected. It's a proportional selection method that promotes diversity.

2. **Tournament Selection:** In this approach, a few `Chromosome`s are randomly selected from the population, and the one with the highest fitness among them becomes a parent. Tournament selection is robust and less biased towards high-fitness `Chromosome`s.

3. **Rank-Based Selection:** `Chromosome`s are ranked by fitness, and selection probabilities are assigned based on these ranks. Rank-based selection can strike a balance between exploration and exploitation and is less sensitive to outliers.

4. **Stochastic Universal Sampling (SUS):** SUS is an enhanced version of roulette wheel selection. It selects multiple parents in a single step by evenly spacing points around the wheel. SUS encourages diversity and can improve convergence.

5. **Boltzmann Selection:** Inspired by thermodynamics, this method assigns selection probabilities based on a temperature parameter. It introduces controlled randomness into the selection process, promoting exploration.

6. **Random Selection:** This technique selects two `Chromosome`s at random.

The choice of a selection technique involves a trade-off between exploration (discovering new areas of the solution space) and exploitation (refining solutions around promising regions). The selection method and parameters directly influence this balance.


In [ ]:
class Selection:

    @staticmethod
    def roulette(population):
        selected_parents = []
        """
            Implement the roulette selection method.
        """
        return selected_parents

    @staticmethod
    def tournament(population, tournament_size):
        selected_parents = []
        """
            Implement the tournament selection method.
        """
        return selected_parents
    

# Feel free to add any further selection methods you please.

#### Crossover (Recombination)
Crossover is a pivotal process within EAs where individuals from the current population are combined to produce offspring with the aim of creating improved solutions. This essential step draws inspiration from genetic recombination in nature and plays a crucial role in driving EAs towards optimal or near-optimal solutions.

Imagine nature's way of mixing genetic material during reproduction. In EAs, crossover mimics this by taking genetic material from two parent individuals and creating one or more offspring with characteristics inherited from both. By blending the traits of successful parents, crossover introduces diversity and promotes the exploration of solution space.

**Common Crossover Techniques:**

1. **One-Point Crossover:** This technique selects a random point along the genetic material of the parents and combines the segments before and after that point. It's a straightforward method that introduces diversity.

2. **Two-Point Crossover:** Similar to one-point crossover, but it selects two random points for crossover. This can lead to more complex recombination of genetic material.

3. **Uniform Crossover:** In this method, each gene (or element) of the offspring is inherited from one of the parents, chosen randomly. It maintains diversity by preserving the genes of both parents.

4. **Arithmetic Crossover:** Applicable when the genetic material represents continuous values, this technique computes a weighted average of the parents' genes to create the offspring. It's commonly used in real-valued optimization problems.

5. **Order Crossover (OX):** Primarily used for permutation-based problems like the Traveling Salesman Problem, OX preserves the order of a subset of genes from one parent and fills the remaining positions with genes from the other parent.

6. **Partially-Mapped Crossover (PMX):** This approach is designed to create offspring by combining genetic material (permutations) from two parent individuals while preserving certain order-based characteristics. It was introduced as an alternative to the Order Crossover (OX) method.

The choice of a crossover technique depends on the problem domain and the characteristics of the solution space. Different techniques strike a balance between exploration (creating diverse solutions) and exploitation (combining promising traits). Tuning crossover rates and parameters can further refine an EA's performance.

In [ ]:
class Crossover:
    """
        Write some crossover operators...
        Remember: Any crossover operators that you write will be required to be added to the EA class (below)
    """

#### Mutation
Mutation is a vital process within EAs where individuals undergo small, random changes to their genetic material. This critical step introduces diversity into the population, fostering exploration and enabling EAs to escape local optima in pursuit of optimal or near-optimal solutions.

Think of mutation as nature's way of introducing genetic novelty into a population. In EAs, mutation mimics this process by randomly altering the genetic material of individuals. While most changes are subtle, occasional mutations can lead to breakthroughs and the discovery of previously unexplored regions of the solution space.

**Common Mutation Techniques:**

1. **Bit Flip Mutation:** Typically used in binary-encoded problems, this method randomly flips individual bits in the genetic representation. It's a simple yet effective way to introduce variability.

2. **Swap Mutation:** For permutation-based problems like the Traveling Salesman Problem (TSP), swap mutation selects two positions at random and swaps the elements at those positions. This preserves the integrity of the permutation.

3. **Inversion Mutation:** In this technique, a randomly selected subset of the genetic material is inverted. In permutation-based problems, this means reversing the order of a portion of the sequence.

4. **Gaussian Mutation:** Commonly employed in real-valued optimization problems, Gaussian mutation adds a small, normally-distributed random value to each gene. It allows for fine-grained adjustments.

5. **Boundary Mutation:** Applicable to problems with defined boundaries, boundary mutation perturbs genes towards the boundary values. It's used to explore extreme regions of the solution space.

The choice of a mutation technique and its parameters depends on the problem domain and the level of diversity needed. Mutation rates and other parameters can be adjusted to control the extent of mutation.

In [ ]:
class Mutation:

    @staticmethod
    def swap(chromosome):
        mutated_chromosome = Chromosome(chromosome)
        """
            Implement the swap mutation operator.
        """
        return chromosome  # Return the original chromosome if no mutation occurs

    # Feel free to add any more that you please below, recalling that you will have to add them to the EA class below.

#### Replacement

Replacement is a pivotal process within EAs that determines which individuals from the current population will make it to the next generation. This crucial step influences the algorithm's ability to preserve diversity, maintain progress, and refine solutions towards optimality.

Think of replacement as nature's way of selecting individuals to continue the evolutionary journey. In EAs, replacement decides which individuals will be retained for the next generation and which will make way for new offspring. It's a balancing act between preserving the best solutions found so far and allowing new genetic material to contribute.

**Common Replacement Techniques:**

1. **Generational Replacement:** In this method, the entire current population is replaced by the offspring generated in the current generation. This approach promotes rapid exploration but may lead to the loss of well-performing individuals.

2. **Elitism:** Elitism ensures that the best-performing individuals from the current generation are directly carried over to the next without any changes. It safeguards the top solutions and prevents them from being lost.

3. **Steady-State Replacement:** In steady-state replacement, a subset of the population is replaced by offspring, while some individuals from the current generation are retained. This maintains a balance between preserving good solutions and introducing diversity.

4. **Age-Based Replacement:** In age-based replacement, individuals in the population are assigned ages, and the oldest individuals are replaced. This approach encourages the retirement of less promising solutions.

The choice of a replacement technique influences the exploration-exploitation trade-off in an EA. Depending on the problem and the desired algorithm behavior, you can tailor the replacement strategy to your specific needs.

In [ ]:
class Replacement: 

    @staticmethod
    def steady_state(population, offspring):
        """
            Implement the steady state replacement method.
        """
        return population

    # Implement other replacement methods as needed

#### Piecing it all Together

Evolutionary algorithms are powerful optimization techniques that mimic the process of natural selection to solve complex problems. They are composed of several essential components, including initialisation, selection, crossover, mutation, and replacement. Here, we'll piece these elements together.

**Initialisation:**
- Begin by creating an initial population of individuals, representing potential solutions to the problem.
- Each individual is generated using a specific initialisation strategy, which can be random, greedy, seed-based, or more.

**Selection:**
- Select individuals from the current population to act as parents for the next generation.
- The selection process can use techniques like roulette wheel, tournament, rank-based, or stochastic universal sampling (SUS).
- High-fitness individuals are more likely to be chosen as parents, promoting the retention of promising traits.

**Crossover:**
- Combine genetic material from selected parents to create offspring.
- Common crossover techniques include one-point, two-point, uniform, arithmetic, and more.
- Crossover introduces diversity and helps explore the solution space.

**Mutation:**
- Apply random changes to the genetic material of individuals.
- Mutation can be implemented using techniques like bit flip, swap, inversion, Gaussian, boundary mutation, and others.
- It introduces variability and allows exploration of new possibilities.

**Replacement:**
- Decide which individuals from the current population will be replaced by offspring.
- Replacement strategies include generational replacement, elitism, steady-state replacement, and age-based replacement.
- Balancing the retention of good solutions with the introduction of diversity is key.

In [ ]:
class EvolutionaryAlgorithm:

    def evolutionary_algorithm() -> Chromosome:
        best = Chromosome()
        global_best = Chromosome()
        population = []
        best_fitnesses = []
        global_best_fitnesses = []
        
        if Parameters.initialisation_method == 'random':
            population = Initialisation.random()
        elif Parameters.initialisation_method == 'augmented':
            population = Initialisation.augmented()
        else:
            print("The initialisation method is invalid!")
            return
        
        for i in range(Parameters.num_generations):
            parents = []

            if Parameters.selection_method == 'roulette':
                parents = Selection.roulette(population)
            elif Parameters.selection_method == 'tournament':
                parents = Selection.tournament(population, 5)
            else:
                print("The selection method is invalid!")
                return
            
            # Add your crossover methods here
            # If you add crossover methods, you will need to implement a new list of chromosomes: children
            # Replace all occurrences of parents[0], parents[1] with children[0] and children[1].
            # Replace the parents in the replacement operators with the children list.
            
            if Parameters.mutation_method == 'swap':
                parents[0] = Mutation.swap(parents[0])
                parents[1] = Mutation.swap(parents[1])
            else:
                print("The mutation method is invalid!")
                return
            
            for c in population: 
                c.evaluate()

            if Parameters.replacement_method == 'steady_state':
                population = Replacement.steady_state(population, parents)
            else:
                print("The replacement method is invalid!")
                return
            
            ordered_pop = sorted(population, key=lambda x: x.fitness)

            best = Chromosome(ordered_pop[0])
            best.evaluate()
            global_best.evaluate()

            if (best.fitness < global_best.fitness) or (global_best.fitness == 0):
                global_best = Chromosome(best)

            best_fitnesses.append(best.fitness)
            global_best_fitnesses.append(global_best.fitness)

            print(f'The fittest chromosome from generation {i} was {best}')
        
        print(f'The fittest chromosome from the evolutionary process was {global_best}!')

        return (best_fitnesses, global_best_fitnesses)


In [ ]:
(best_fitnesses, global_best_fitnesses) = EvolutionaryAlgorithm.evolutionary_algorithm()

xs = [i for i in range(0,Parameters.num_generations)]

plt.plot(xs, best_fitnesses, label='Line 1', linestyle='-', color='blue')
plt.plot(xs, global_best_fitnesses, label='Line 2', linestyle='--',color='red')

plt.show()

### Thinking Time
+ What was your overall experience with implementing an evolutionary algorithm from scratch?

+ Reflect on the problem in this tutorial. How well did the evolutionary algorithm perform in finding solutions? Were there any unexpected challenges or successes?

+ Consider the components of an evolutionary algorithm, such as initialisation, selection, crossover, mutation, and replacement. Which component did you find most challenging to implement, and why?

+ How did you decide on the parameters for your evolutionary algorithm, such as population size, mutation rate, and termination criteria? Did you experiment with different values, and if so, what did you learn from those experiments?

+ In your implementation, did you encounter any issues related to convergence, diversity, or premature convergence? How did you address these issues?

+ Did you incorporate any specialized techniques or operators, such as elitism, local search, or adaptive strategies, into your evolutionary algorithm? How did these enhancements impact the algorithm's performance?

+ What lessons did you learn about the importance of parameter tuning, balancing exploration and exploitation, and maintaining diversity within the population?

+ Consider the real-world applicability of the problem you addressed in your project. How could your evolutionary algorithm be extended or adapted for practical use in various domains?

+ Reflect on the software tools or libraries you used for your implementation. How did these tools assist or challenge you in the development of your algorithm?

+ In terms of coding and implementation skills, what areas did you find most beneficial to develop during this project? Are there specific programming or algorithmic concepts that you gained a deeper understanding of?

+ Looking ahead, how might you improve or extend your evolutionary algorithm for future projects or real-world applications? Are there additional features or strategies you would like to explore?

+ Consider the teamwork, collaboration, or peer review aspects of your project. How did interactions with your peers or instructors contribute to your learning and project outcomes?

+ Finally, reflect on your overall growth in understanding evolutionary algorithms and their applications. How has this course and project shaped your perspective on optimisation and problem-solving?